In [1]:
import numpy as np 
import scipy.io as sio
import os, os.path
import sys
import tensorflow as tf
sys.path.append('../')
import re
from datasets.dataset_utils import int64_feature, float_feature, bytes_feature ,ImageCoder, norm

from PIL import Image

In [2]:
data_path = '../../TextBoxes-TensorFlow/data/sythtext/'
cellname = 'gt'
textname = 'txt'
imcell = 'imnames'
wordname = 'wordBB'
charname = 'charBB'
NUMoffolder = 200

In [11]:
def _convert_to_example(image_data, shape, charbb, label,imname):

	#print 'shape: {}, height:{}, width:{}'.format(shape,shape[0],shape[1])
	example = tf.train.Example(features=tf.train.Features(feature={
			'image/height': int64_feature(shape[0]),
			'image/width': int64_feature(shape[1]),
			'image/channels': int64_feature(shape[2]),
			'image/shape': int64_feature(shape),
			'image/object/bbox/x0': float_feature(charbb[0,0,:].tolist()),
			'image/object/bbox/x1': float_feature(charbb[0,1,:].tolist()),
			'image/object/bbox/x2': float_feature(charbb[0,2,:].tolist()),
			'image/object/bbox/x3': float_feature(charbb[0,3,:].tolist()),
			'image/object/bbox/y0': float_feature(charbb[1,0,:].tolist()),
			'image/object/bbox/y1': float_feature(charbb[1,1,:].tolist()),
			'image/object/bbox/y2': float_feature(charbb[1,2,:].tolist()),
			'image/object/bbox/y3': float_feature(charbb[1,3,:].tolist()),
			'image/object/bbox/label': int64_feature(label),
			'image/format': bytes_feature('jpeg'),
			'image/encoded': bytes_feature(image_data),
			'image/name': bytes_feature(imname.tostring()),
			}))
	return example
	

def _processing_image(charbb, imname,coder):
	image_data = tf.gfile.GFile(data_path+imname, 'r').read()
	image = coder.decode_jpeg(image_data)
	shape = image.shape
	numofbox = charbb.shape[2]
	#in the order [x1,x2,x3,x4,y1,y2,y3,y4]
	label = [1 for i in range(numofbox)]
	shape = list(shape)
	return image_data, shape, charbb, label, imname


In [4]:
labels = sio.loadmat(data_path + 'gt.mat')

In [5]:
print labels.keys()
texts = labels[textname]
imnames = labels[imcell]
wordBB = labels[wordname]
charBB = labels[charname]

['wordBB', 'txt', '__header__', '__globals__', '__version__', 'imnames', 'charBB']


In [ ]:
dir = 0+1
pattern = re.compile(r'^{}\/'.format(dir))
res =[k for k in range(imnames.shape[1]) if pattern.match(imnames[0,k][0]) != None ]

In [ ]:
image = np.array(Image.open(imnames[0,res[1]][0]))
image.shape

In [ ]:
chars = charBB[0,res[1]]

In [ ]:
for i in range(N):
    chars[0,:,i] = chars[0,:,i]/image.shape[1]
    chars[1,:,i] = chars[1,:,i]/image.shape[0]
    
# 2 dimention, first is x, second is y

In [ ]:
print chars[:,:,0].tolist()
print chars[:,:,0].reshape(-1).tolist()
print chars[0,1,:]

In [ ]:
import cv2
fig = plt.figure(figsize=(10,10))
for i in range(chars.shape[2]):
    cv2.polylines(image,np.array(np.expand_dims(chars[:,:,i].T,axis=0),dtype=np.int32),True ,255)
plt.imshow(image)
plt.show()

In [12]:
coder = ImageCoder()
for i in range(NUMoffolder):
    tf_filename = str(i+1) + '.tfrecord'
    tfrecord_writer = tf.python_io.TFRecordWriter('../data/'+tf_filename)
    dir = i+1
    pattern = re.compile(r'^{}\/'.format(dir))
    print dir
    print pattern
    res =[k for k in range(imnames.shape[1]) if pattern.match(imnames[0,k][0]) != None ]
    print "The size of %s folder : %s" % (dir,len(res))
    # shuffle
    res = np.random.permutation(res)
    for j in res:
        charbb = charBB[0,j]
        imname = imnames[0,j][0]
        #print str(i) + imname
        image_data, shape, bbox, label ,imname= _processing_image(charbb, imname,coder)

        example = _convert_to_example(image_data, shape, bbox, label, imname)
        tfrecord_writer.write(example.SerializeToString())  
print 'Transform to tfrecord finished'
tf.boo

1
The size of 1 folder : 4262
2
The size of 2 folder : 4449


NotFoundError: ../../TextBoxes-TensorFlow/data/sythtext/2/ant+hill_86_98.jpg